In [1]:
%%html
<style>
table {float: left}
</style>

# Semesterarbeit Statistische Datenanalyse

## Dokumenteninformationen


| Titel   <img width=200/> |      todo     |
|:---------|:------------- |
| Schule |  Fernfachhochschule Schweiz |
| Studiengang |    Certificate of Advanced Studies in Statistische Datenanalyse & Datenvisualisierung   |
| Kennung | DS-C-SD002.StatDa.ZH-Sa-1.PVA.FS23 |
| Semester | Frühlingssemester 2023 |
| Dozent | **Peter Tellenbach**<br>peter.tellenbach@ffhs.ch<br> |
| Autor | **Patrick Hirschi**<br>Geburtsdatum: 12.01.1990<br>Matrikelnummer: 10-179-026<br>Studierenden-ID: 200768<br>patrick-hirschi@gmx.ch<br> |

## Datenbeschaffung

### Nutzungsbedingungen

Sehr wichtig ist, dass die Nutzungsbedingungen der Datensets beachtet werden. Es gibt verschiedene Kategorien:
  * **OPEN** (offen für alle Zwecke, Quellenangabe empfohlen)
  * **OPEN BY** (offen für alle Zwecke, Quellenangabe verpflichtend)
  * **OPEN ASK** (nicht-kommerziell OK, kommerziell muss erfragt werden beim Datenlieferanten, Quellenangabe empfohlen)
  * **OPEN BY ASK** (nicht-kommerziell OK, kommerziell muss erfragt werden beim Datenlieferanten, Quellenangabe verpflichtend)
  
Sämtliche für dieses Projekt verwendeten Datensets sind in der Kategorie **OPEN** und stammen alle von derselben Quelle ([opendata.swiss](https://opendata.swiss/de/dataset/stundlich-aktualisierte-luftqualitatsmessungen-seit-19831)).

In [14]:
# import required modules
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import sys
import os
import glob
import datetime
import requests
from requests.exceptions import HTTPError
import pandas as pd
import numpy as np

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go

# print versions
print('matplotlib: %s' % matplotlib.__version__)
print('requests: %s' % requests.__version__)
print('pandas: %s' % pd.__version__)
print('numpy: %s' % np.__version__)
print('plotly: %s' % plotly.__version__)

# log success
print(f'The module import was successful!')

matplotlib: 3.6.3
requests: 2.28.2
pandas: 1.5.3
numpy: 1.24.2
plotly: 5.13.0
The module import was successful!


### Konfiguration

Die Daten sind über URLs eindeutig beschrieben, und können so einfach neu heruntergeladen werden. Der Export ist konfigurierbar und kann auch ausgeschalten werden. Bei jedem Export werden die bereits vorhandenen Datenfiles mit einem Zeitstempel versehen und archiviert. Dies garantiert die vollständige Reproduzierbarkeit.
  
Wenn zusätzliche Datenfiles von opendata.swiss geladen werden sollen, so können die entsprechenden URLs und Filenamen ganz einfach dazukonfiguriert werden, und der Code fürs Laden der Daten muss nicht weiter angefasst werden.

In [12]:
# switch to True for reloading data or switch to False if wanting to skip 
# the reload and work with the existing data
download_new_source_data = False
# assign directory
directory = './data'
# get current datetime
datetime_now = datetime.datetime.now()
# base url
base_url = 'https://data.stadt-zuerich.ch/dataset/ugz_luftschadstoffmessung_stundenwerte/download/'
# start year
start_year = 1983
# end year
end_year = 2023

### Laden der Daten von opendata.swiss

Wenn ein (neues) Laden der Daten konfiguriert wurde, wird in einem ersten Schritt aufgeräumt. Alle vorhandenen Datenfiles im ./data Verzeichnis werden dann ins ./data/archive Verzeichnis verschoben.
  
Das python requests Modul ermöglicht es mit nur einer Zeile den entsprechenden Datensatz herunterzuladen. In der Folge wird das neue File im lokalen Filesystem erstellt.
  
Ein selektives Neuladen von nur einzelnen Files ist nicht vorgesehen, wäre aber mit paar wenigen Code-Anpassungen möglich.

In [13]:
# datetime method for logging purposes
def get_current_time_str():
    return datetime.datetime.now().strftime("%H:%M:%S.%f")

# only execute this code if the switch "download_new_source_data" is set to true
if download_new_source_data:
    # iterate over old data files in the directory and archive in ./data/archive folder
    for file in os.listdir(directory):
        # join the filepath information
        fullpath = os.path.join(directory, file)
        # split the filepath into filename and fileextension
        filename = os.path.splitext(file)[0]
        fileextension = os.path.splitext(file)[1]
        # generate a datestring to add to the archived filename
        datestring = datetime_now.strftime("%Y%m%d%H%M%S") + '_'
        # checking if it is a CSV/XLSX file to avoid archiving non-data files
        if os.path.isfile(fullpath) and (fullpath.endswith('.csv') or fullpath.endswith('.xlsx')):
            print(f'{get_current_time_str()}: Found file with path: {fullpath}')
            # generate archive filename
            newpath = directory + '/archive/' + datestring + filename + fileextension
            print(f'{get_current_time_str()}: Archive filepath will be: {newpath}')
            # rename the file and move it directly to the archive directory
            os.rename(fullpath,newpath)
            print(f'{get_current_time_str()}: Successfully archived file {fullpath}'
                  f' to {newpath}!') 
            
    # download latest data files from opendata.swiss
    for x in range(start_year, end_year+1):
        try:
            # build url
            url = base_url + f'ugz_ogd_air_h1_{str(x)}.csv'
            # access the data file url
            response = requests.get(url, allow_redirects=True)          
            # write content to the target file
            open(os.path.join(directory, f'ugz_ogd_air_h1_{str(x)}.csv'), 'wb').write(response.content)          
            # get file stats
            file_stats = os.stat(os.path.join(directory, f'ugz_ogd_air_h1_{str(x)}.csv'))
            # If the response was successful, no Exception will be raised
            response.raise_for_status()
        except HTTPError as http_err:
            # log the details of the HTTP exception (specific catch)
            print(f'{get_current_time_str()}: HTTP error occurred while' 
                  f'downloading data file {key} from {url}: {err}') 
        except Exception as err:
            # log the details of any other exception (generic catch)
            print(f'{get_current_time_str()}: Other error occurred while'
                  f'downloading data file {key} from {url}: {err}')  
        else:
            # download and file write was succesful
            print(f'{get_current_time_str()}: Successfully loaded '
                  f'{file_stats.st_size} bytes from {url} into the data file ugz_ogd_air_h1_{str(x)}.csv!') 
else:
    # source data was intentionally not re-loaded
    print(f'{get_current_time_str()}: '
          f'No data was reloaded from source! If this was not intended, change the switch '
          f'download_new_source_data to True.') 

18:12:31.319201: No data was reloaded from source! If this was not intended, change the switch download_new_source_data to True.


In [15]:
# Get CSV files list from a folder
csv_files = glob.glob(directory + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
df_air_quality = pd.concat(df_list, ignore_index=True)

In [17]:
df_air_quality.head(5)

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
0,1987-01-01T03:00+0100,Zch_Stampfenbachstrasse,CO,h1,mg/m3,0.10,bereinigt
1,1987-01-01T03:00+0100,Zch_Stampfenbachstrasse,NO2,h1,µg/m3,33.42,bereinigt
2,1987-01-01T03:00+0100,Zch_Stampfenbachstrasse,NO,h1,µg/m3,6.18,bereinigt
3,1987-01-01T03:00+0100,Zch_Stampfenbachstrasse,NOx,h1,ppb,22.43,bereinigt
4,1987-01-01T03:00+0100,Zch_Stampfenbachstrasse,O3,h1,µg/m3,39.90,bereinigt


## Datenaufbereitung

In [18]:
df_air_quality.Standort.unique()

array(['Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse',
       'Zch_Heubeeribüel', 'Zch_Rosengartenstrasse'], dtype=object)

In [19]:
df_air_quality.Status.unique()

array(['bereinigt', 'provisorisch'], dtype=object)

In [20]:
df_air_quality.Intervall.unique()

array(['h1'], dtype=object)

In [21]:
df_air_quality.Parameter.unique()

array(['CO', 'NO2', 'NO', 'NOx', 'O3', 'SO2', 'PM10', 'PM2.5'],
      dtype=object)